In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [3]:
now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [4]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [7]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [11]:
x_train.shape

(60000, 28, 28)

In [8]:
# create complete model
model = Sequential(feature_layers + classification_layers)

In [9]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 16s 523us/step - loss: 0.1630 - acc: 0.9487 - val_loss: 0.0341 - val_acc: 0.9868
Epoch 2/5
30596/30596 [==============================] - 3s 91us/step - loss: 0.0435 - acc: 0.9859 - val_loss: 0.0133 - val_acc: 0.9955
Epoch 3/5
30596/30596 [==============================] - 3s 91us/step - loss: 0.0313 - acc: 0.9900 - val_loss: 0.0084 - val_acc: 0.9971
Epoch 4/5
30596/30596 [==============================] - 3s 91us/step - loss: 0.0244 - acc: 0.9922 - val_loss: 0.0097 - val_acc: 0.9967
Epoch 5/5
30596/30596 [==============================] - 3s 91us/step - loss: 0.0223 - acc: 0.9934 - val_loss: 0.0074 - val_acc: 0.9973
Training time: 0:00:27.612466
Test score: 0.007430394905449469
Test accuracy: 0.9972757345787118


In [10]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 2s 68us/step - loss: 0.2265 - acc: 0.9359 - val_loss: 0.0542 - val_acc: 0.9827
Epoch 2/5
29404/29404 [==============================] - 2s 54us/step - loss: 0.0723 - acc: 0.9776 - val_loss: 0.0326 - val_acc: 0.9893
Epoch 3/5
29404/29404 [==============================] - 2s 53us/step - loss: 0.0548 - acc: 0.9827 - val_loss: 0.0257 - val_acc: 0.9912
Epoch 4/5
29404/29404 [==============================] - 2s 53us/step - loss: 0.0446 - acc: 0.9867 - val_loss: 0.0251 - val_acc: 0.9916
Epoch 5/5
29404/29404 [==============================] - 2s 53us/step - loss: 0.0395 - acc: 0.9885 - val_loss: 0.0231 - val_acc: 0.9920
Training time: 0:00:08.564398
Test score: 0.02305627352317163
Test accuracy: 0.9919769593630031
